In [1]:
from pyspark import SparkContext
from pyspark.sql import SparkSession, SQLContext
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, FloatType, Row

In [2]:
spark = SparkContext(master='local', appName='DataFrames')
sqlContext = SQLContext(spark)

22/10/01 18:44:07 WARN Utils: Your hostname, debian resolves to a loopback address: 127.0.1.1; using 192.168.100.4 instead (on interface eno2)
22/10/01 18:44:07 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
22/10/01 18:44:07 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
path_file = '/home/jeo/Documents/platzi/curso_de_fundamentos_de_spark_para_big_data/curso-apache-spark-platzi-master/files/'

In [4]:
def eliminar_encabezado(indice, iterador):
    return iter(list(iterador)[1:])

In [5]:
# False in the schema indicates that it is required
juegoSchema = StructType([
    StructField('juego_id', IntegerType(), False),
    StructField('nombre_juego', StringType(), False),
    StructField('anio', StringType(), False),
    StructField('temporada', StringType(), False),
    StructField('ciudad', StringType(), False)
])

In [6]:
juegoDataFrame = sqlContext.read.schema(juegoSchema) \
                .option('header', 'true').csv(path_file + 'juegos.csv')

In [7]:
juegoDataFrame.show(5)

+--------+------------+----+---------+---------+
|juego_id|nombre_juego|anio|temporada|   ciudad|
+--------+------------+----+---------+---------+
|       1| 1896 Verano|1896|   Verano|   Athina|
|       2| 1900 Verano|1900|   Verano|    Paris|
|       3| 1904 Verano|1904|   Verano|St. Louis|
|       4| 1906 Verano|1906|   Verano|   Athina|
|       5| 1908 Verano|1908|   Verano|   London|
+--------+------------+----+---------+---------+
only showing top 5 rows



22/10/01 18:44:11 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , nombre_juego, annio, temporada, ciudad
 Schema: juego_id, nombre_juego, anio, temporada, ciudad
Expected: juego_id but found: 
CSV file: file:///home/jeo/Documents/platzi/curso_de_fundamentos_de_spark_para_big_data/curso-apache-spark-platzi-master/files/juegos.csv


###### create DF from RDD

In [8]:
resultado = spark.textFile(path_file+'resultados.csv') \
            .map(lambda l: l.split(','))

In [9]:
resultadoGanador = resultado.filter(lambda l: 'NA' not in l[1])

In [47]:
deportistaOlimpicoRDD = spark.textFile(path_file+'deportista.csv') \
                        .map(lambda l: l.split(","))
deportistaOlimpicoRDD2 = spark.textFile(path_file+'deportista2.csv') \
                        .map(lambda l: l.split(","))

In [11]:
equiposOlimpicosRDD = spark.textFile(path_file+'paises.csv') \
                        .map(lambda line: line.split(','))

In [48]:
deportistaOlimpicoRDD = deportistaOlimpicoRDD.union(deportistaOlimpicoRDD2)

In [13]:
resultado = spark.textFile(path_file+'resultados.csv') \
            .map(lambda l: l.split(','))

In [25]:
equipoDeportistaOlimpico = deportistaOlimpicoRDD.map(lambda l: (l[-1], l[:-1])) \
                            .join(equiposOlimpicosRDD.map(lambda x: (x[0], x[2])))

In [15]:
paisesMedallas = equipoDeportistaOlimpico.map(lambda l: (l[1][0][0], (l[0], l[1][0][1:], l[1][1]))) \
    .join(resultadoGanador.map(lambda x: (x[2], x[1])))

In [27]:
equipoDeportistaOlimpico.take(1)[0][0]

'199'

In [21]:
schema_equipo_deportista_olimpico = StructType([
    StructField('id_equipo', IntegerType(), False),
    StructField('deportista_id', IntegerType(), False),
    StructField('nombre', StringType(), False),
    StructField('genero', IntegerType(), False),
    StructField('edad', IntegerType(), False),
    StructField('altura', IntegerType(), False),
    StructField('peso', IntegerType(), False),
    StructField('pais', StringType(), False)
])

In [28]:
equipoDeportistaOlimpico = equipoDeportistaOlimpico.map(lambda l: (
    int(l[0]),
    int(l[1][0][0]),
    str(l[1][0][1]),
    int(l[1][0][2]),
    int(l[1][0][3]),
    int(l[1][0][4]),
    int(l[1][0][5]),
    str(l[1][1])
))

In [29]:
equipoDeportistaOlimpico.take(1)

[(199, 1, 'A Dijiang', 1, 24, 180, 80, 'CHN')]

In [30]:
equipoDeportistaOlimpicoDF = sqlContext.createDataFrame(equipoDeportistaOlimpico, schema_equipo_deportista_olimpico)
equipoDeportistaOlimpicoDF.show(5)

+---------+-------------+-------------+------+----+------+----+----+
|id_equipo|deportista_id|       nombre|genero|edad|altura|peso|pais|
+---------+-------------+-------------+------+----+------+----+----+
|      199|            1|    A Dijiang|     1|  24|   180|  80| CHN|
|      199|            2|     A Lamusi|     1|  23|   170|  60| CHN|
|      199|          602|Abudoureheman|     1|  22|   182|  75| CHN|
|      199|         1463|    Ai Linuer|     1|  25|   160|  62| CHN|
|      199|         1464|    Ai Yanhan|     2|  14|   168|  54| CHN|
+---------+-------------+-------------+------+----+------+----+----+
only showing top 5 rows



In [52]:
deportistaOlimpicoRDD.take(1)[0][0]

'deportista_id'

In [53]:
deportistaOlimpicoRDD = deportistaOlimpicoRDD.mapPartitionsWithIndex(eliminar_encabezado)

In [54]:
deportistaOlimpicoRDD.take(1)

[['1', 'A Dijiang', '1', '24', '180', '80', '199']]

In [55]:
deportistaOlimpicoRDD = deportistaOlimpicoRDD.map(lambda l: (
    int(l[0]),
    str(l[1]),
    int(l[2]),
    int(l[3]),
    int(l[4]),
    float(l[5]),
    int(l[6])
))

In [56]:
deportistaOlimpicoRDD.take(1)

[(1, 'A Dijiang', 1, 24, 180, 80.0, 199)]

In [57]:
schema = StructType([
    StructField('deportista_id', IntegerType(), False),
    StructField('nombre', StringType(), False),
    StructField('genero', IntegerType(), False),
    StructField('edad', IntegerType(), False),
    StructField('altura', IntegerType(), False),
    StructField('peso', FloatType(), False),
    StructField('equipo_id', IntegerType(), False)
])

In [58]:
deportistaDF = sqlContext.createDataFrame(deportistaOlimpicoRDD, schema)
deportistaDF.show(5)

+-------------+--------------------+------+----+------+----+---------+
|deportista_id|              nombre|genero|edad|altura|peso|equipo_id|
+-------------+--------------------+------+----+------+----+---------+
|            1|           A Dijiang|     1|  24|   180|80.0|      199|
|            2|            A Lamusi|     1|  23|   170|60.0|      199|
|            3| Gunnar Nielsen Aaby|     1|  24|     0| 0.0|      273|
|            4|Edgar Lindenau Aabye|     1|  34|     0| 0.0|      278|
|            5|Christine Jacoba ...|     2|  21|   185|82.0|      705|
+-------------+--------------------+------+----+------+----+---------+
only showing top 5 rows



In [59]:
paisesMedallas = paisesMedallas.map(lambda l: (
    int(l[0]),
    int(l[1][0][0]),
    str(l[1][0][1][0]),
    int(l[1][0][1][1]),
    int(l[1][0][1][2]),
    int(l[1][0][1][3]),
    int(l[1][0][1][4]),
    str(l[1][0][2]),
    str(l[1][1])
))

In [60]:
schema_paises_medallas = StructType([
    StructField('deportista_id', IntegerType(), False),
    StructField('id_equipo', IntegerType(), False),
    StructField('nombre', StringType(), False),
    StructField('genero', IntegerType(), False),
    StructField('edad', IntegerType(), False),
    StructField('altura', IntegerType(), False),
    StructField('peso', IntegerType(), False),
    StructField('pais', StringType(), False),
    StructField('medalla', StringType(), False)
])

In [61]:
paisesMedallasDF = sqlContext.createDataFrame(paisesMedallas, schema_paises_medallas)
paisesMedallasDF.show(5)

+-------------+---------+---------------+------+----+------+----+----+-------+
|deportista_id|id_equipo|         nombre|genero|edad|altura|peso|pais|medalla|
+-------------+---------+---------------+------+----+------+----+----+-------+
|         7597|      199|   Bao Yingying|     2|  24|   172|  67| CHN| Silver|
|        17282|      199|     Cai Huijue|     2|  16|   174|  63| CHN| Bronze|
|        17996|      199|   Cao Mianying|     2|  21|   176|  71| CHN| Silver|
|        19779|      199|       Chang Si|     2|  25|   170|  56| CHN| Silver|
|        19791|      199|Chang Yongxiang|     1|  24|   178|  74| CHN| Silver|
+-------------+---------+---------------+------+----+------+----+----+-------+
only showing top 5 rows



In [62]:
equiposOlimpicosRDD = equiposOlimpicosRDD.mapPartitionsWithIndex(eliminar_encabezado)

In [63]:
equiposOlimpicosRDD.take(5)

[['1', '30. Februar', 'AUT'],
 ['2', 'A North American Team', 'MEX'],
 ['3', 'Acipactli', 'MEX'],
 ['4', 'Acturus', 'ARG'],
 ['5', 'Afghanistan', 'AFG']]

In [64]:
equiposOlimpicosRDD = equiposOlimpicosRDD.map(lambda l: (
    int(l[0]),
    str(l[1]),
    str(l[2])
))

In [65]:
schema_equipos_olimpicos = StructType([
    StructField('id_equipo', IntegerType(), False),
    StructField('nombre_equipo', StringType(), False),
    StructField('sigla', StringType(), False)
])

In [66]:
equiposOlimpicosDF = sqlContext.createDataFrame(equiposOlimpicosRDD, schema_equipos_olimpicos)
equiposOlimpicosDF.show(5)

+---------+--------------------+-----+
|id_equipo|       nombre_equipo|sigla|
+---------+--------------------+-----+
|        1|         30. Februar|  AUT|
|        2|A North American ...|  MEX|
|        3|           Acipactli|  MEX|
|        4|             Acturus|  ARG|
|        5|         Afghanistan|  AFG|
+---------+--------------------+-----+
only showing top 5 rows



In [67]:
deportesOlimpicosSchemaRDD = StructType([
    StructField('deporte_id', IntegerType(), False),
    StructField('deporte', StringType(), False)
])
deportesDF = sqlContext.read.schema(deportesOlimpicosSchemaRDD).option("header", "True").csv(path_file + "deporte.csv")
deportesDF.show(5)

+----------+-------------+
|deporte_id|      deporte|
+----------+-------------+
|         1|   Basketball|
|         2|         Judo|
|         3|     Football|
|         4|   Tug-Of-War|
|         5|Speed Skating|
+----------+-------------+
only showing top 5 rows



In [68]:
eventoSchemaRDD = StructType([
    StructField('evento_id', IntegerType(), False),
    StructField('evento', StringType(), False),
    StructField('deporte_id', IntegerType(), False)
])
eventoDF = sqlContext.read.schema(eventoSchemaRDD).option('header', 'True').csv(path_file + 'evento.csv')
eventoDF.show(5)

+---------+--------------------+----------+
|evento_id|              evento|deporte_id|
+---------+--------------------+----------+
|        1|Basketball Men's ...|         1|
|        2|Judo Men's Extra-...|         2|
|        3|Football Men's Fo...|         3|
|        4|Tug-Of-War Men's ...|         4|
|        5|Speed Skating Wom...|         5|
+---------+--------------------+----------+
only showing top 5 rows



In [69]:
resultadoSchemaRDD = StructType([
    StructField('resultado_id', IntegerType(), False),
    StructField('medalla', StringType(), False),
    StructField('deportista_id', IntegerType(), False),
    StructField('juego_id', IntegerType(), False),
    StructField('evento_id', IntegerType(), False)
])
resultadoDF = sqlContext.read.schema(resultadoSchemaRDD).option('header', 'True').csv(path_file + 'resultados.csv')
resultadoDF.show(5)

+------------+-------+-------------+--------+---------+
|resultado_id|medalla|deportista_id|juego_id|evento_id|
+------------+-------+-------------+--------+---------+
|           1|     NA|            1|      39|        1|
|           2|     NA|            2|      49|        2|
|           3|     NA|            3|       7|        3|
|           4|   Gold|            4|       2|        4|
|           5|     NA|            5|      36|        5|
+------------+-------+-------------+--------+---------+
only showing top 5 rows



In [70]:
deportesDF.printSchema()

root
 |-- deporte_id: integer (nullable = true)
 |-- deporte: string (nullable = true)



In [ ]:
deportistaDF.printSchema()

In [ ]:
deportistaDF = deportistaDF.withColumnRenamed('genero', 'sexo').drop('altura')
deportistaDF.printSchema()

In [ ]:
from pyspark.sql.functions import *

In [ ]:
deportistaDF = deportistaDF.select('deportista_id', 'nombre', col('edad').alias('edadAlJugar'), 'equipo_id')
deportistaDF.show(5)

In [ ]:
deportistaDF = deportistaDF.filter(
    (deportistaDF.edadAlJugar != 0))

In [ ]:
deportistaDF.sort('edadAlJugar').show(5)

In [ ]:
spark

In [ ]:
spark.stop()